# SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

 **Задания**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def sql_look(table):    
    display(pd.io.sql.read_sql('''SELECT * FROM {} LIMIT 5'''.format(table), con = engine))
    display()

**books**

In [3]:
sql_look('books')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**authors**

In [4]:
sql_look('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**publishers**

In [5]:
sql_look('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**ratings**

In [6]:
sql_look('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**reviews**

In [7]:
sql_look('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Выводы**

- подключились к postqre
- ознакомились с 5 таблицами: books,authors,publishers,ratings,reviews

## Задания

**1.Посчитайте, сколько книг вышло после 1 января 2000 года;**

In [8]:
query = '''
            SELECT COUNT(*) AS count_of_books
FROM books 
WHERE publication_date > '2000-01-01';
        '''
pd.io.sql.read_sql(query, con = engine)

,count_of_books
0,819


С 2 января 2020 вышла 819 книг

**2.Для каждой книги посчитайте количество обзоров и среднюю оценку;**

In [9]:
query = '''
SELECT 
    books.title AS book_title,
    books.book_id AS book_id,
    COUNT(DISTINCT reviews.review_id) AS review_count, 
    AVG(ratings.rating) AS avg_rating
FROM 
    books
    LEFT JOIN reviews ON books.book_id = reviews.book_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY 
    books.title,
    books.book_id
ORDER BY
        review_count DESC;
    
      '''
pd.io.sql.read_sql(query, con = engine)

,book_title,book_id,review_count,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Water for Elephants,963,6,3.977273
2,The Glass Castle,734,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
996,The Natural Way to Draw,808,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Essential Tales and Poems,221,0,4.000000


**3.Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;**

In [10]:
query = '''
            SELECT publishers.publisher, 
                   COUNT(*) AS books_count
            FROM books
            JOIN publishers ON books.publisher_id = publishers.publisher_id
            WHERE num_pages > 50
            GROUP BY publishers.publisher
            ORDER BY books_count DESC
            LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher,books_count
0,Penguin Books,42


**4.Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;**

In [11]:
query = '''
SELECT 
  a.author,
  AVG(r.rating) as avg_rating
FROM 
  authors a 
  JOIN books b ON b.author_id = a.author_id
  JOIN ratings r ON b.book_id = r.book_id
WHERE 
  b.book_id IN (
    SELECT 
      r.book_id 
    FROM 
      ratings r 
    GROUP BY 
      r.book_id 
    HAVING 
      COUNT(*) >= 50
  )
GROUP BY 
  a.author
ORDER BY 
  avg_rating DESC 
LIMIT 1;
        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**5.Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.**

In [12]:
query = '''
  SELECT AVG (reviews_count) as avg_reviews_count
  FROM ( 
        SELECT username,
        COUNT(review_id) as reviews_count
        FROM reviews
        GROUP BY username
        HAVING username IN 
        (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT (rating_id) > 48)                  
        ) AS r
        '''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews_count
0,24.0


## Выводы

- После 2000-01-01 выпущено 819 книг
- Посчитали количество обзоров - 999, среднюю оценку для каждой из книг
- Наибольшее число книг - 42, выпустило издательство Penguin Books
- 4.28 - самая высокая оценка у J.K. Rowling/Mary GrandPré
- 24 - Среднее количество обзоров от пользователей, которые поставили больше 48 оценок 